In [ ]:
# conda install -c conda-forge ta-lib

In [ ]:
!pip uninstall numpy pandas -y

In [ ]:
conda install numpy=1.21 pandas=1.3

In [ ]:
!pip install pandas-ta tuneta awswrangler 

In [ ]:
!find ~/.cache/pip -type d -name "__pycache__" -exec rm -r {} +

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tuneta.tune_ta import TuneTA
import awswrangler as wr
import pandas_ta as pta
import talib as tta

In [2]:
model_data = pd.read_parquet(r"s3://sg-ds-project/alpha-generator/training_data_transform/model_data_2005_20240ct.parquet")

In [3]:
pd.set_option('display.max_columns',None)
model_data.head(3)

date    open    high     low   close   volume nse_symbol  adj_close  \
0 2020-07-13  237.50  246.00  231.31  236.78  51840.0     360ONE     236.78   
1 2020-07-14  242.00  242.00  232.75  236.76  30676.0     360ONE     236.76   
2 2020-07-15  232.65  238.19  232.00  232.64  52360.0     360ONE     232.64   

   next_day_return_close  target_close_intraday  next_day_return_high  \
0                  -2.17                      0                  0.00   
1                  -0.00                      0                  2.38   
2                  -0.88                      0                  1.41   

   target_high_intraday  next_day_return_low  target_low_intraday  \
0                     0                -3.82                    1   
1                     1                -0.28                    0   
2                     0                -3.29                    1   

   3d_return_high  target_high_3d  target_high_3d_multiclass  ABER_ZG_14_15  \
0        0.000000             0.0                        0.0     251.450000   
1        2.381259             0.0                        0.0     250.138571   
2        1.657435             0.0                        0.0     247.400476   

   ABER_SG_14_15  ABER_XG_14_15  ABER_ATR_14_15    ACCBL_10  ACCBM_10  \
0     266.157252     236.742748       14.707252  217.039809   246.120   
1     264.482007     235.795136       14.343435  221.973420   244.252   
2     261.200349     233.600603       13.799873  221.587363   242.541   

     ACCBU_10            AD    ADOSC_12_26  AMATe_LR_10_20_2  \
0  283.399809  3.710045e+06 -145826.095708                 0   
1  272.338420  3.705966e+06 -145854.983907                 0   
2  268.802363  3.664433e+06 -147528.604320                 0   

   AMATe_SR_10_20_2   AO_5_34        OBV  OBV_min_20  OBV_max_30  \
0                 1  0.728235  3165728.0   2530340.0   3653152.0   
1                 1 -2.133647  3135052.0   2530340.0   3653152.0   
2                 1 -6.119941  3082692.0   2530340.0   3653152.0   

        OBVs_12       OBVs_26  AOBV_LR_2  AOBV_SR_2  APO_12_26     ATRe_5  \
0  3.293386e+06  2.957463e+06          1          0  -1.602244  12.449026   
1  3.277590e+06  2.973301e+06          0          1  -2.409744  11.809221   
2  3.261986e+06  2.987570e+06          0          1  -3.191410  10.685377   

    BBL_5_2.0  BBM_5_2.0   BBU_5_2.0  BBB_5_2.0  BBP_5_2.0  BIAS_SMA_14  \
0  234.663851    243.128  251.592149   6.962710   0.125007    -0.049305   
1  233.120738    240.774  248.427262   6.357216   0.237759    -0.045526   
2  230.750887    237.966  245.181113   6.063986   0.130914    -0.051525   

        AR_14  CDL_2CROWS  CDL_3BLACKCROWS  CDL_3INSIDE  CDL_3LINESTRIKE  \
0  116.293205         0.0              0.0          0.0              0.0   
1   98.811500         0.0              0.0          0.0              0.0   
2   58.611401         0.0              0.0          0.0              0.0   

   CDL_3OUTSIDE  CDL_3STARSINSOUTH  CDL_3WHITESOLDIERS  CDL_ABANDONEDBABY  \
0           0.0                0.0                 0.0                0.0   
1           0.0                0.0                 0.0                0.0   
2           0.0                0.0                 0.0                0.0   

   CDL_ADVANCEBLOCK  CDL_BELTHOLD  CDL_BREAKAWAY  CDL_CLOSINGMARUBOZU  \
0               0.0           0.0            0.0                  0.0   
1               0.0        -100.0            0.0                  0.0   
2               0.0           0.0            0.0                  0.0   

   CDL_CONCEALBABYSWALL  CDL_COUNTERATTACK  CDL_DARKCLOUDCOVER  \
0                   0.0                0.0                 0.0   
1                   0.0                0.0                 0.0   
2                   0.0                0.0                 0.0   

   CDL_DOJI_10_0.1  CDL_DOJISTAR  CDL_DRAGONFLYDOJI  CDL_ENGULFING  \
0            100.0           0.0                0.0            0.0   
1              0.0           0.0                0.0        

In [4]:
model_data = model_data[model_data['date']>='2018-01-01']

In [5]:
model_data.shape

(691861, 423)

In [6]:
model_data['date'].min(),model_data['date'].max()

(Timestamp('2018-01-01 00:00:00'), Timestamp('2024-10-15 00:00:00'))

In [7]:
model_data['nse_symbol'].nunique()

478

In [8]:
pd.set_option('display.max_columns',None)
model_data.head(1)

date   open   high     low   close   volume nse_symbol  adj_close  \
0 2020-07-13  237.5  246.0  231.31  236.78  51840.0     360ONE     236.78   

   next_day_return_close  target_close_intraday  next_day_return_high  \
0                  -2.17                      0                   0.0   

   target_high_intraday  next_day_return_low  target_low_intraday  \
0                     0                -3.82                    1   

   3d_return_high  target_high_3d  target_high_3d_multiclass  ABER_ZG_14_15  \
0             0.0             0.0                        0.0         251.45   

   ABER_SG_14_15  ABER_XG_14_15  ABER_ATR_14_15    ACCBL_10  ACCBM_10  \
0     266.157252     236.742748       14.707252  217.039809    246.12   

     ACCBU_10            AD    ADOSC_12_26  AMATe_LR_10_20_2  \
0  283.399809  3.710045e+06 -145826.095708                 0   

   AMATe_SR_10_20_2   AO_5_34        OBV  OBV_min_20  OBV_max_30  \
0                 1  0.728235  3165728.0   2530340.0   3653152.0   

        OBVs_12       OBVs_26  AOBV_LR_2  AOBV_SR_2  APO_12_26     ATRe_5  \
0  3.293386e+06  2.957463e+06          1          0  -1.602244  12.449026   

    BBL_5_2.0  BBM_5_2.0   BBU_5_2.0  BBB_5_2.0  BBP_5_2.0  BIAS_SMA_14  \
0  234.663851    243.128  251.592149    6.96271   0.125007    -0.049305   

        AR_14  CDL_2CROWS  CDL_3BLACKCROWS  CDL_3INSIDE  CDL_3LINESTRIKE  \
0  116.293205         0.0              0.0          0.0              0.0   

   CDL_3OUTSIDE  CDL_3STARSINSOUTH  CDL_3WHITESOLDIERS  CDL_ABANDONEDBABY  \
0           0.0                0.0                 0.0                0.0   

   CDL_ADVANCEBLOCK  CDL_BELTHOLD  CDL_BREAKAWAY  CDL_CLOSINGMARUBOZU  \
0               0.0           0.0            0.0                  0.0   

   CDL_CONCEALBABYSWALL  CDL_COUNTERATTACK  CDL_DARKCLOUDCOVER  \
0                   0.0                0.0                 0.0   

   CDL_DOJI_10_0.1  CDL_DOJISTAR  CDL_DRAGONFLYDOJI  CDL_ENGULFING  \
0            100.0           0.0                0.0            0.0   

   CDL_EVENINGDOJISTAR  CDL_EVENINGSTAR  CDL_GAPSIDESIDEWHITE  \
0                  0.0              0.0                   0.0   

   CDL_GRAVESTONEDOJI  CDL_HAMMER  CDL_HANGINGMAN  CDL_HARAMI  \
0                 0.0         0.0             0.0         0.0   

   CDL_HARAMICROSS  CDL_HIGHWAVE  CDL_HIKKAKE  CDL_HIKKAKEMOD  \
0              0.0        -100.0          0.0             0.0   

   CDL_HOMINGPIGEON  CDL_IDENTICAL3CROWS  CDL_INNECK  CDL_INSIDE  \
0               0.0                  0.0         0.0           0   

   CDL_INVERTEDHAMMER  CDL_KICKING  CDL_KICKINGBYLENGTH  CDL_LADDERBOTTOM  \
0                 0.0          0.0                  0.0               0.0   

   CDL_LONGLEGGEDDOJI  CDL_LONGLINE  CDL_MARUBOZU  CDL_MATCHINGLOW  \
0               100.0           0.0           0.0              0.0   

   CDL_MATHOLD  CDL_MORNINGDOJISTAR  CDL_MORNINGSTAR  CDL_ONNECK  \
0          0.0                  0.0              0.0         0.0   

   CDL_PIERCING  CDL_RICKSHAWMAN  CDL_RISEFALL3METHODS  CDL_SEPARATINGLINES  \
0           0.0            100.0                   0.0                  0.0   

   CDL_SHOOTINGSTAR  CDL_SHORTLINE  CDL_SPINNINGTOP  CDL_STALLEDPATTERN  \
0               0.0            0.0           -100.0                 0.0   

   CDL_STICKSANDWICH  CDL_TAKURI  CDL_TASUKIGAP  CDL_THRUSTING  CDL_TRISTAR  \
0                0.0         0.0            0.0            0.0          0.0   

   CDL_UNIQUE3RIVER  CDL_UPSIDEGAP2CROWS  CDL_XSIDEGAP3METHODS    CMF_20  \
0               0.0                  0.0                   0.0 -0.277382   

      CG_14  CKSPl_10_1.0_9  CKSPs_10_1.0_9     CMO_14  COPC_11_14_10  DEC_5  \
0 -7.598132      283.244992      245.528901 -12.271748      -5.240936      1   

      DEMA_10     DMP_14     DMN_14  DCL_20_20  DCM_20_20  DCU_20_20  \
0  241.668817  54.731345  27.348707     231.31     265.03     298.75   

       EMA_20   ENTP_20  EOM_14_100000000  BULLP_14   BEARP_14  FISHE

In [9]:
import os
os.cpu_count()

48

In [10]:
model_data['date'] = pd.to_datetime(model_data['date'])

train_df = model_data[model_data['date'] < '2023-01-01']
test_df = model_data[model_data['date'] >= '2023-01-03']

train_df['date'] = pd.to_datetime(train_df['date'])

if 'nse_symbol' in train_df.index.names:
    train_df.reset_index(inplace=True, drop=False)  # Use drop=False to prevent dropping the existing index

if 'level_0' in train_df.columns:
    train_df.drop(columns='level_0', inplace=True)

train_df.set_index('date', inplace=True)

assert pd.api.types.is_datetime64_any_dtype(train_df.index), "Date index is not of type datetime"

print(train_df.index)
print(train_df.index.dtype)

# Initialize and fit TuneTA (now the index is only the datetime 'date')
tt = TuneTA(n_jobs=48, verbose=True)
tt.fit(train_df[['open', 'high', 'low', 'close', 'volume']], train_df['target_high_intraday'],
    indicators=[
                'tta.SMA', 'tta.EMA', 'tta.DEMA', 
#                  'tta.ADX', 'tta.MACD', 'tta.KAMA', 'tta.BBANDS', 'tta.RSI', 'tta.MFI', 'tta.CCI', 'pta.apo', 'pta.ao',
#                 'pta.adosc', 'pta.aberration', 'pta.accbands',
#                  'pta.aobv', 'pta.amat', 'tta.ROC', 'pta.alma', 'tta.AROON', 'pta.bias', 'pta.cg', 'pta.atr', 'pta.cmf', 
#                 'pta.cdl', 'pta.cti', 'pta.coppock', 'pta.cksp', 'pta.chop', 'pta.brar',
#                  'pta.cmo', 'pta.eri', 'pta.dm', 'pta.donchian', 'pta.fwma', 'pta.fisher', 'pta.efi', 'pta.eom',
# #                  'pta.hilo', 'pta.entropy', 'pta.er', 'pta.ebsw', 'pta.natr', 'pta.kst', 'pta.kc', 'pta.massi',
#                  'pta.psar', 'pta.percent_return', 'pta.ichimoku', 'pta.ppo', 'pta.log_return', 'pta.median',
#                  'pta.pwma', 'pta.linreg', 'pta.pvo', 'pta.kdj', 'pta.pgo', 'pta.increasing', 'pta.mad',
#                  'pta.kurtosis', 'pta.mom', 'pta.pdist', 'pta.psl', 'pta.inertia', 'pta.vidya', 'pta.trix',
#                  'pta.supertrend', 'pta.sinwma', 'pta.trima', 'pta.swma', 'pta.zlma', 'pta.vwma', 'pta.tema',
#                  'pta.ssf', 'pta.t3', 'pta.tsi', 'pta.zscore', 'pta.stdev', 'pta.vhf',
#                  'pta.squeeze_pro','pta.slope', 'pta.ttm_trend', 'pta.willr', 'pta.uo', 'pta.vortex',
#                  'pta.stochrsi', 'pta.skew', 'pta.thermo',
#                  'pta.qstick', 'pta.rsx', 'pta.rvi', 'pta.qqe', 'pta.rvgi'
               ],
    ranges=[(3, 20),(20,53)],
    trials=100,
    early_stop=20,
)

DatetimeIndex(['2020-07-13', '2020-07-14', '2020-07-15', '2020-07-16',
               '2020-07-17', '2020-07-20', '2020-07-21', '2020-07-22',
               '2020-07-23', '2020-07-24',
               ...
               '2022-12-19', '2022-12-20', '2022-12-21', '2022-12-22',
               '2022-12-23', '2022-12-26', '2022-12-27', '2022-12-28',
               '2022-12-29', '2022-12-30'],
              dtype='datetime64[ns]', name='date', length=488540, freq=None)
datetime64[ns]


[I 2024-10-30 05:17:43,253] A new study created in memory with name: tta.SMA(X.close, timeperiod=trial.suggest_int('timeperiod', 3, 20), )
[I 2024-10-30 05:17:43,329] A new study created in memory with name: tta.EMA(X.close, timeperiod=trial.suggest_int('timeperiod', 3, 20), )
[I 2024-10-30 05:17:43,391] A new study created in memory with name: tta.EMA(X.close, timeperiod=trial.suggest_int('timeperiod', 20, 53), )
[I 2024-10-30 05:17:43,410] A new study created in memory with name: tta.DEMA(X.close, timeperiod=trial.suggest_int('timeperiod', 3, 20), )
[I 2024-10-30 05:17:43,466] A new study created in memory with name: tta.DEMA(X.close, timeperiod=trial.suggest_int('timeperiod', 20, 53), )
[I 2024-10-30 05:17:43,467] A new study created in memory with name: tta.SMA(X.close, timeperiod=trial.suggest_int('timeperiod', 20, 53), )
[I 2024-10-30 05:19:15,725] Trial 0 finished with value: 0.07504298216591421 and parameters: {'timeperiod': 32}. Best is trial 0 with value: 0.07504298216591421.

[I 2024-10-30 05:31:07,203] Trial 7 finished with value: 0.07662896362562445 and parameters: {'timeperiod': 18}. Best is trial 1 with value: 0.07666701581373446.
[I 2024-10-30 05:31:08,768] Trial 7 finished with value: 0.07665189680829136 and parameters: {'timeperiod': 18}. Best is trial 1 with value: 0.07666521860210022.
[I 2024-10-30 05:31:20,977] Trial 8 finished with value: 0.07528136530652972 and parameters: {'timeperiod': 40}. Best is trial 2 with value: 0.07546979856020979.
[I 2024-10-30 05:32:12,286] Trial 8 finished with value: 0.07576134318279441 and parameters: {'timeperiod': 40}. Best is trial 1 with value: 0.07673294249352455.
[I 2024-10-30 05:32:13,606] Trial 8 finished with value: 0.07572190160339068 and parameters: {'timeperiod': 40}. Best is trial 6 with value: 0.0766082491799873.
[I 2024-10-30 05:32:34,784] Trial 8 finished with value: 0.07543058653632709 and parameters: {'timeperiod': 13}. Best is trial 6 with value: 0.07625708316029269.
[I 2024-10-30 05:32:47,348] T

[I 2024-10-30 05:44:41,001] Trial 17 finished with value: 0.075055759061315 and parameters: {'timeperiod': 49}. Best is trial 2 with value: 0.07546979856020979.
[I 2024-10-30 05:44:48,353] Trial 16 finished with value: 0.07614482777327285 and parameters: {'timeperiod': 45}. Best is trial 10 with value: 0.07682649949251903.
[I 2024-10-30 05:45:19,388] Trial 16 finished with value: 0.07666521860210022 and parameters: {'timeperiod': 20}. Best is trial 10 with value: 0.07666521860210022.
[I 2024-10-30 05:46:00,899] Trial 16 finished with value: 0.07593824773757973 and parameters: {'timeperiod': 11}. Best is trial 6 with value: 0.07625708316029269.
[I 2024-10-30 05:46:09,686] Trial 16 finished with value: 0.07666701581373446 and parameters: {'timeperiod': 20}. Best is trial 1 with value: 0.07666701581373446.
[I 2024-10-30 05:46:11,777] Trial 16 finished with value: 0.07666521860210022 and parameters: {'timeperiod': 20}. Best is trial 1 with value: 0.07666521860210022.
[I 2024-10-30 05:46:13

[I 2024-10-30 05:59:27,360] Trial 24 finished with value: 0.07601911583913669 and parameters: {'timeperiod': 10}. Best is trial 17 with value: 0.07631195675527659.
[I 2024-10-30 05:59:36,480] Trial 27 finished with value: 0.07528136530652972 and parameters: {'timeperiod': 40}. Best is trial 18 with value: 0.075525718927474.
[I 2024-10-30 06:00:00,506] Trial 25 finished with value: 0.0762237161104583 and parameters: {'timeperiod': 26}. Best is trial 10 with value: 0.07666521860210022.
[I 2024-10-30 06:00:38,224] Trial 26 finished with value: 0.07627265507472752 and parameters: {'timeperiod': 43}. Best is trial 10 with value: 0.07682649949251903.
[I 2024-10-30 06:01:07,007] Trial 25 finished with value: 0.07631195675527659 and parameters: {'timeperiod': 6}. Best is trial 17 with value: 0.07631195675527659.
[I 2024-10-30 06:01:07,363] Trial 28 finished with value: 0.07530460942392252 and parameters: {'timeperiod': 34}. Best is trial 18 with value: 0.075525718927474.
[I 2024-10-30 06:01:38

In [11]:
tt.fit_times()

    Indicator      Times
--  -----------  -------
 3  tta.DEMA     3995.82
 5  tta.DEMA     3670.02
 2  tta.EMA      3222.33
 4  tta.SMA      3140.82
 0  tta.EMA      2312.46
 1  tta.SMA      2308.55


In [12]:
tt.report(target_corr=True, features_corr=False)


Indicator Correlation to Target:

                          Correlation
----------------------  -------------
tta_EMA_timeperiod_17        0.076653
tta_SMA_timeperiod_18        0.076629
tta_EMA_timeperiod_26        0.076224
tta_DEMA_timeperiod_5        0.076187
tta_SMA_timeperiod_44        0.076102
tta_DEMA_timeperiod_43       0.075463


In [ ]:
# pta_natr_length_5_mamode_trima                                                         0.211861
# pta_chop_length_3_atr_length_26                                                        0.111366
# pta_bias_length_5_mamode_fwma                                                          0.087264
# pta_cmf_length_22                                                                      0.0813
# tta_ROC_timeperiod_25                                                                  0.080392
# pta_coppock_length_16_fast_7_slow_27                                                   0.079891
# pta_kst_roc1_20_roc2_12_roc3_4_roc4_11_sma1_12_sma2_23_sma3_20_sma4_27_signal_16       0.079754
# pta_kc_length_6_mamode_tema                                                            0.079707
# pta_donchian_lower_length_20_upper_length_23                                           0.079171
# pta_accbands_length_5_mamode_rma                                                       0.07917
# pta_massi_fast_13_slow_28                                                              0.078355
# pta_cksp_p_5_q_10                                                                      0.077012
# pta_alma_length_17                                                                     0.077006
# pta_hilo_high_length_9_low_length_20_mamode_pwma                                       0.077002
# pta_aberration_length_20_atr_length_23                                                 0.076664
# tta_KAMA_timeperiod_12                                                                 0.076367
# pta_fwma_length_6                                                                      0.076207
# tta_BBANDS_timeperiod_6                                                                0.074462
# pta_entropy_length_25                                                                  0.073208
# tta_MFI_timeperiod_22                                                                  0.07186
# pta_cg_length_22                                                                       0.070832
# pta_brar_length_27                                                                     0.070161
# tta_ADX_timeperiod_27                                                                  0.059742
# pta_cmo_length_28                                                                      0.053732
# tta_RSI_timeperiod_28                                                                  0.053732
# pta_amat_fast_19_slow_29_lookback_14_mamode_rma                                        0.05222
# pta_atr_length_28_mamode_ema                                                           0.051057
# pta_dm_length_28_mamode_ema                                                            0.044351
# tta_CCI_timeperiod_28                                                                  0.043299
# pta_fisher_length_27_signal_5                                                          0.04058
# pta_cti_length_22                                                                      0.040143
# pta_adosc_fast_28_slow_26                                                              0.03675
# pta_ao_fast_22_slow_17                                                                 0.034168
# pta_apo_fast_17_slow_27_mamode_pwma                                                    0.034065
# pta_efi_length_22_mamode_sma                                                           0.032216
# tta_MACD_fastperiod_22_slowperiod_26_signalperiod_10                                   0.031794
# pta_er_length_14                                                                       0.030151
# pta_eri_length_6                                                                       0.022898
# pta_eom_length_25                                                                      0.018059
# tta_AROON_timeperiod_13                                                                0.015051
# pta_aobv_fast_18_slow_29_max_lookback_22_min_lookback_22_mamode_sinwma                 0.009474
# pta_ebsw_length_23                                                                     0.002523
# pta_trix_length_26_signal_8                       0.085236
# pta_percent_return_length_25                      0.080392
# pta_psar                                          0.078855
# pta_ichimoku_tenkan_27_kijun_29_senkou_15         0.077933
# pta_supertrend_length_6                           0.077346
# pta_median_length_22                              0.076601
# pta_sinwma_length_18                              0.076583
# pta_swma_length_19                                0.076542
# pta_trima_length_19                               0.076542
# pta_zlma_length_5                                 0.076299
# pta_pwma_length_10                                0.07628
# pta_linreg_length_7                               0.076251
# pta_log_return_length_22                          0.076209
# pta_vwma_length_19                                0.076165
# pta_tema_length_22                                0.075092
# pta_vidya_length_23                               0.074692
# pta_ppo_fast_5_slow_11_signal_21_mamode_ema       0.0745
# pta_pvo_fast_3_slow_29_signal_26                  0.068992
# pta_pgo_length_27                                 0.050531
# pta_kdj_length_27_signal_27                       0.047158
# pta_increasing_length_22                          0.04299
# pta_mad_length_27                                 0.039704
# pta_kurtosis_length_25                            0.029984
# pta_mom_length_19                                 0.026936
# pta_pdist                                         0.025576
# pta_psl_length_17                                 0.020859
# pta_inertia_length_8_rvi_length_8                 0.010835
# pta_ssf_length_27                                                                      0.076121
# pta_t3_length_10                                                                       0.075116
# pta_qqe_length_24_smooth_21                                                            0.064321
# pta_tsi_fast_28_slow_30_signal_23_mamode_rma                                           0.058811
# pta_qstick_length_27                                                                   0.05352
# pta_rvgi_length_24_swma_length_7                                                       0.052843
# pta_rsx_length_27                                                                      0.04829
# pta_zscore_length_28                                                                   0.043504
# pta_stdev_length_27_ddof_6                                                             0.039295
# pta_squeeze_pro_bb_length_9_kc_length_26_mom_length_18_mom_smooth_18_mamode_fwma       0.034672
# pta_vortex_length_22                                                                   0.033769
# pta_ttm_trend_length_25                                                                0.032002
# pta_willr_length_28                                                                    0.030744
# pta_vhf_length_23                                                                      0.028586
# pta_slope_length_19                                                                    0.026936
# pta_stochrsi_length_4_rsi_length_8_k_23_d_18_mamode_linreg                             0.020673
# pta_uo_fast_25_medium_25_slow_25                                                       0.020517
# pta_skew_length_28                                                                     0.018849
# pta_rvi_length_7                                                                       0.017797
# pta_thermo_length_22_long_14_short_8_mamode_sinwma                                     0.01057
# tta_EMA_timeperiod_17        0.076653
# tta_SMA_timeperiod_18        0.076629
# tta_EMA_timeperiod_26        0.076224
# tta_DEMA_timeperiod_5        0.076187
# tta_SMA_timeperiod_44        0.076102
# tta_DEMA_timeperiod_43       0.075463